In [1]:
import os
import time
import numpy as np
import cv2
from collections import deque
from scipy.fft import fft
import csv
from scipy.signal import find_peaks

# Configurable parameters
THRESHOLD_BINARY = 240
IGNORE_TOP_PIXELS = 125
MIN_CONTOUR_AREA = 600
MAX_CONTOUR_AREA = 1500  # Maximum contour area threshold
MIN_DISTANCE_BETWEEN_CONTOURS = 10
MAX_AREA_RATIO_DIFF = 0.7
BUFFER_MAXLEN = 20
DISAPPEARANCE_THRESHOLD = 10
BORDER_THICKNESS = 5
TOP_BORDER_PERCENTAGE = 0.30  # Ignore top 30% of the frame
SIDE_BORDER_PERCENTAGE = 0.30  # Ignore 30% from each side of the frame
# Coordinates for the area to crop (x1, y1: top-left, x2, y2: bottom-right)
CROP_X1, CROP_Y1 = 50, 50
CROP_X2, CROP_Y2 = 600, 600
# Resize dimensions (width, height)
RESIZE_WIDTH, RESIZE_HEIGHT = 600, 600

MIN_CIRCULARITY = 0.7  # Minimum circularity to consider a shape as round

# Initialize a global unique ID counter
next_id = 0

def convert_to_binary(frame, threshold=THRESHOLD_BINARY):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Calculate the region to be ignored based on the percentages
    height, width = gray.shape
    ignore_top_pixels = int(height * TOP_BORDER_PERCENTAGE)
    ignore_side_pixels = int(width * SIDE_BORDER_PERCENTAGE)
    
    # Set ignored regions to zero
    gray[:ignore_top_pixels, :] = 0  # Ignore the top pixels
    gray[:, :ignore_side_pixels] = 0  # Ignore left side
    gray[:, width-ignore_side_pixels:] = 0  # Ignore right side
    
    _, binary = cv2.threshold(gray, threshold, 255, cv2.THRESH_BINARY)
    return binary

def merge_close_contours(contours, min_distance=MIN_DISTANCE_BETWEEN_CONTOURS, max_area_ratio_diff=MAX_AREA_RATIO_DIFF):
    merged_contours = []
    used = [False] * len(contours)

    def get_centroid(contour):
        M = cv2.moments(contour)
        if M['m00'] == 0:
            return None
        return (int(M['m10'] / M['m00']), int(M['m01'] / M['m00']))

    for i, c1 in enumerate(contours):
        if used[i]:
            continue
        merged = [c1]
        centroid1 = get_centroid(c1)
        if centroid1 is None:
            continue
        area1 = cv2.contourArea(c1)
        for j, c2 in enumerate(contours):
            if i != j and not used[j]:
                centroid2 = get_centroid(c2)
                if centroid2 is None:
                    continue
                dist = np.linalg.norm(np.array(centroid1) - np.array(centroid2))
                area2 = cv2.contourArea(c2)
                area_ratio_diff = abs(area1 - area2) / max(area1, area2)

                if dist < min_distance and area_ratio_diff < max_area_ratio_diff:
                    merged.append(c2)
                    used[j] = True
        merged_contours.append(np.vstack(merged))
        used[i] = True
    return merged_contours

def extract_contours(binary_frame):
    contours, _ = cv2.findContours(binary_frame, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = merge_close_contours(contours)
    return contours

def frame_objects(contours, frame_shape):
    objects = []
    frame_height, frame_width = frame_shape

    for contour in contours:
        contour_area = cv2.contourArea(contour)
        if MIN_CONTOUR_AREA < contour_area < MAX_CONTOUR_AREA:
            rect = cv2.minAreaRect(contour)
            box = cv2.boxPoints(rect)
            box = box.astype(int)
            center, size, angle = rect
            if angle == 0:
                angle = 90

            if any(point[0] <= BORDER_THICKNESS or point[1] <= BORDER_THICKNESS or 
                   point[0] >= frame_width - BORDER_THICKNESS or 
                   point[1] >= frame_height - BORDER_THICKNESS for point in box):
                continue

            rect = (center, size, angle)
            objects.append((rect, box, contour))
    return objects

def save_object_parameters(objects):
    all_features = []
    for rect, _, _ in objects:
        center, size, _ = rect
        width, height = size
        area = width * height

        features = [area]
        all_features.append(features)
    return all_features


def get_object_key(center, object_buffers, last_known_centers, threshold=MIN_DISTANCE_BETWEEN_CONTOURS):
    for key, last_center in last_known_centers.items():
        if np.linalg.norm(np.array(center) - np.array(last_center)) < threshold:
            return key
    return None

def draw_contours(frame, objects, object_buffers, frame_counter, last_seen_frame, last_known_centers):
    max_len = BUFFER_MAXLEN
    fps = 10  # Frames per second

    for rect, box, _ in objects:
        # Draw the bounding box (box) on the frame
        cv2.drawContours(frame, [box], 0, (0, 255, 0), 2)

        # Draw window progress bar
        center = rect[0]  # Center of the object
        key = get_object_key(center, object_buffers, last_known_centers)  # Pass last_known_centers here

        if key is not None:
            buffer_len = len(object_buffers[key])

            # Calculate the position for the progress bar
            x, y = int(center[0]), int(center[1])
            bar_height = 10
            bar_width = 50

            # Draw the background of the bar
            cv2.rectangle(frame, (x, y - 30), (x + bar_width, y - 30 + bar_height), (0, 0, 0), -1)
            
            # Draw the progress
            progress = int((buffer_len / max_len) * bar_width)
            cv2.rectangle(frame, (x, y - 30), (x + progress, y - 30 + bar_height), (0, 255, 0), -1)

            # Draw the text indicator
            cv2.putText(frame, f"Window: {buffer_len}/{max_len}", (x, y - 35), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)

            # Display the actual window values in rows of 3
            window_values = [f"{value:.2f}" for value in object_buffers[key]]  # Display the area values
            for i in range(0, len(window_values), 3):
                row_values = " | ".join(window_values[i:i+3])
                cv2.putText(frame, row_values, (x, y - 50 - (i//3) * 15), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)

            # Perform FFT and find the dominant frequency if the buffer is full
            if buffer_len == max_len:
                func = np.array(object_buffers[key])
                fftx = np.abs(np.fft.rfft(func))
                f_axis = np.linspace(0, fps/2, int(0.5 * func.size) + 1)
                Y1 = fftx / np.max(fftx)
                x_fft = Y1[1:]
                peaks, _ = find_peaks(x_fft, height=np.max(x_fft) * 0.5)  # Finding peaks above 50% of the max

                if len(peaks) > 0:
                    dominant_frequency = f_axis[peaks[0] + 1]  # Peaks[0] is the index in the frequency domain
                    # Display the dominant frequency on the frame
                    cv2.putText(frame, f"Freq: {dominant_frequency:.2f} Hz", (x+10, y +5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

    # Display windows even for objects that have disappeared but are within the threshold
    for key, buffer in object_buffers.items():
        if len(buffer) > 0:
            # Draw window for objects not currently detected but within the threshold
            if key not in objects and frame_counter - last_seen_frame[key] <= DISAPPEARANCE_THRESHOLD:
                center = last_known_centers[key]
                x, y = int(center[0]), int(center[1])
                bar_height = 10
                bar_width = 50

                # Draw the background of the bar
                cv2.rectangle(frame, (x, y - 30), (x + bar_width, y - 30 + bar_height), (0, 0, 0), -1)
                
                # Draw the progress
                progress = int((len(buffer) / max_len) * bar_width)
                cv2.rectangle(frame, (x, y - 30), (x + progress, y - 30 + bar_height), (0, 255, 0), -1)

                # Draw the text indicator
                cv2.putText(frame, f"Window: {len(buffer)}/{max_len}", (x, y - 35), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)

                # Display the actual window values in rows of 3
                window_values = [f"{value:.2f}" for value in buffer]  # Display the area values
                for i in range(0, len(window_values), 3):
                    row_values = " | ".join(window_values[i:i+3])
                    cv2.putText(frame, row_values, (x, y - 50 - (i//3) * 15), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)

def process_frame(frame, frame_counter, object_buffers, last_seen_frame, last_known_centers):
    global next_id
    binary_frame = convert_to_binary(frame)
    contours = extract_contours(binary_frame)
    objects = frame_objects(contours, frame.shape[:2])

    features = save_object_parameters(objects)

    seen_keys = set()
    if features:  # Check if features is not empty
        for obj_features, obj in zip(features, objects):
            area = obj_features[0]
            center = obj[0][0]  # Extract the center from the object
            key = get_object_key(center, object_buffers, last_known_centers)
            if key is None:
                key = next_id
                next_id += 1
                object_buffers[key] = deque(maxlen=BUFFER_MAXLEN)
                # Create a new CSV file for the new object
                with open(f'C:\\Users\\thaim\\OneDrive\\Desktop\\Tal_Projects\\Gas_detector\\General_Codes\\Gas_Detector_5_24\\LED_FREQUENCY_TEST\\window_{key}.csv', 'w', newline='') as csvfile:
                    csvwriter = csv.writer(csvfile)
                    csvwriter.writerow([f'Frame {frame_counter}'] + [area])  # First row with the first value
            else:
                # Save the current state of the deque for each frame, even if it's not full
                with open(f'C:\\Users\\thaim\\OneDrive\\Desktop\\Tal_Projects\\Gas_detector\\General_Codes\\Gas_Detector_5_24\\LED_FREQUENCY_TEST\\window_{key}.csv', 'a', newline='') as csvfile:
                    csvwriter = csv.writer(csvfile)
                    csvwriter.writerow([f'Frame {frame_counter}'] + list(object_buffers[key]))
                
            object_buffers[key].append(area)  # Append only the area
            last_seen_frame[key] = frame_counter
            last_known_centers[key] = center  # Update the last known center
            seen_keys.add(key)

    # Add zero for missing objects up to DISAPPEARANCE_THRESHOLD
    for key in list(object_buffers.keys()):
        if key not in seen_keys:
            frames_missing = frame_counter - last_seen_frame[key]
            if frames_missing <= DISAPPEARANCE_THRESHOLD:
                object_buffers[key].append(0)  # Add zero if the object is missing
                # Update CSV file with the new zero value
                with open(f'C:\\Users\\thaim\\OneDrive\\Desktop\\Tal_Projects\\Gas_detector\\General_Codes\\Gas_Detector_5_24\\LED_FREQUENCY_TEST\\window_{key}.csv', 'a', newline='') as csvfile:
                    csvwriter = csv.writer(csvfile)
                    csvwriter.writerow([f'Frame {frame_counter}'] + list(object_buffers[key]))
            else:
                # The buffer is not deleted immediately; it's up to draw_contours to handle display logic.
                pass

    return object_buffers, objects, binary_frame
def crop_and_resize(frame, x1, y1, x2, y2, resize_width, resize_height):
    # Crop the area based on provided coordinates
    cropped_frame = frame[y1:y2, x1:x2]
    
    # Resize the cropped area
    resized_frame = cv2.resize(cropped_frame, (resize_width, resize_height), interpolation=cv2.INTER_LINEAR)
    
    return resized_frame

def main(video_source):
    time.sleep(0.1)
    cap = cv2.VideoCapture(video_source)
    if not cap.isOpened():
        print("Error: Could not open video source.")
        return

    frame_counter = 0
    object_buffers = {}  # Dictionary to store buffers for each object
    last_seen_frame = {}  # Dictionary to store the last frame an object was seen
    last_known_centers = {}  # Dictionary to store the last known centers of objects

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        time.sleep(0.1)
        frame_counter += 1
        object_buffers, objects, binary_frame = process_frame(frame, frame_counter, object_buffers, last_seen_frame, last_known_centers)  # Capture binary_frame
        
        # Analyze the buffers to find any frequency patterns
        # analyze_buffers(object_buffers)
        
        # Draw the contours and the window indicators
        draw_contours(frame, objects, object_buffers, frame_counter, last_seen_frame, last_known_centers)  # Pass last_known_centers here

        # Crop and resize the specific area based on user input
        cropped_resized_frame = crop_and_resize(frame, CROP_X1, CROP_Y1, CROP_X2, CROP_Y2, RESIZE_WIDTH, RESIZE_HEIGHT)

        # Display the cropped and resized frame
        cv2.imshow('Cropped and Resized Frame', binary_frame)

        # Display the real-time frames      
        cv2.imshow('Real-time Object Detection', frame)


        # Display the binary frame
        #cv2.imshow('Binary Frame', binary_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()


In [2]:
if __name__ == "__main__":
    video_path = r"C:\Users\thaim\OneDrive\Desktop\Leds for simulator\3m - full shutter - 449  - trim.mp4"
    main(video_path)  # Use 0 for webcam, or provide a video file path

Error: Could not open video source.


### TEST


In [3]:
import os
import time
import numpy as np
import cv2
from collections import deque
from scipy.fft import fft
import csv
from scipy.signal import find_peaks

# Configurable parameters
THRESHOLD_BINARY = 160
IGNORE_TOP_PIXELS = 125
MIN_CONTOUR_AREA = 1
MAX_CONTOUR_AREA = 2500  # Maximum contour area threshold
MIN_DISTANCE_BETWEEN_CONTOURS = 10
MAX_AREA_RATIO_DIFF = 0.7
BUFFER_MAXLEN = 20
DISAPPEARANCE_THRESHOLD = 10
BORDER_THICKNESS = 5
TOP_BORDER_PERCENTAGE = 0.30  # Ignore top 30% of the frame
SIDE_BORDER_PERCENTAGE = 0.20  # Ignore 20% from each side of the frame
MIN_CIRCULARITY = 0.7  # Minimum circularity to consider a shape as round

# Initialize a global unique ID counter
next_id = 0

def convert_to_binary(frame, threshold=THRESHOLD_BINARY):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Calculate the region to be ignored based on the percentages
    height, width = gray.shape
    ignore_top_pixels = int(height * TOP_BORDER_PERCENTAGE)
    ignore_side_pixels = int(width * SIDE_BORDER_PERCENTAGE)
    
    # Set ignored regions to zero
    gray[:ignore_top_pixels, :] = 0  # Ignore the top pixels
    gray[:, :ignore_side_pixels] = 0  # Ignore left side
    gray[:, width-ignore_side_pixels:] = 0  # Ignore right side
    
    _, binary = cv2.threshold(gray, threshold, 255, cv2.THRESH_BINARY)
    return binary

def merge_close_contours(contours, min_distance=MIN_DISTANCE_BETWEEN_CONTOURS, max_area_ratio_diff=MAX_AREA_RATIO_DIFF):
    merged_contours = []
    used = [False] * len(contours)

    def get_centroid(contour):
        M = cv2.moments(contour)
        if M['m00'] == 0:
            return None
        return (int(M['m10'] / M['m00']), int(M['m01'] / M['m00']))

    for i, c1 in enumerate(contours):
        if used[i]:
            continue
        merged = [c1]
        centroid1 = get_centroid(c1)
        if centroid1 is None:
            continue
        area1 = cv2.contourArea(c1)
        for j, c2 in enumerate(contours):
            if i != j and not used[j]:
                centroid2 = get_centroid(c2)
                if centroid2 is None:
                    continue
                dist = np.linalg.norm(np.array(centroid1) - np.array(centroid2))
                area2 = cv2.contourArea(c2)
                area_ratio_diff = abs(area1 - area2) / max(area1, area2)

                if dist < min_distance and area_ratio_diff < max_area_ratio_diff:
                    merged.append(c2)
                    used[j] = True
        merged_contours.append(np.vstack(merged))
        used[i] = True
    return merged_contours

def extract_contours(binary_frame):
    contours, _ = cv2.findContours(binary_frame, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = merge_close_contours(contours)
    return contours

def frame_objects(contours, frame_shape):
    objects = []
    frame_height, frame_width = frame_shape

    for contour in contours:
        contour_area = cv2.contourArea(contour)
        perimeter = cv2.arcLength(contour, True)
        if perimeter > 0:
            circularity = 4 * np.pi * contour_area / (perimeter * perimeter)
        else:
            circularity = 0

        if MIN_CONTOUR_AREA < contour_area < MAX_CONTOUR_AREA and circularity >= MIN_CIRCULARITY:
            rect = cv2.minAreaRect(contour)
            box = cv2.boxPoints(rect)
            box = box.astype(int)
            center, size, angle = rect
            if angle == 0:
                angle = 90

            if any(point[0] <= BORDER_THICKNESS or point[1] <= BORDER_THICKNESS or 
                   point[0] >= frame_width - BORDER_THICKNESS or 
                   point[1] >= frame_height - BORDER_THICKNESS for point in box):
                continue

            rect = (center, size, angle)
            objects.append((rect, box, contour))
    return objects

def save_object_parameters(objects):
    all_features = []
    for rect, _, _ in objects:
        center, size, _ = rect
        width, height = size
        area = width * height

        features = [area]
        all_features.append(features)
    return all_features


def get_object_key(center, object_buffers, last_known_centers, threshold=MIN_DISTANCE_BETWEEN_CONTOURS):
    for key, last_center in last_known_centers.items():
        if np.linalg.norm(np.array(center) - np.array(last_center)) < threshold:
            return key
    return None

def draw_contours(frame, objects, object_buffers, frame_counter, last_seen_frame, last_known_centers):
    max_len = BUFFER_MAXLEN
    fps = 10  # Frames per second

    for rect, box, _ in objects:
        # Draw the bounding box (box) on the frame
        cv2.drawContours(frame, [box], 0, (0, 255, 0), 2)

        # Draw window progress bar
        center = rect[0]  # Center of the object
        key = get_object_key(center, object_buffers, last_known_centers)  # Pass last_known_centers here

        if key is not None:
            buffer_len = len(object_buffers[key])

            # Calculate the position for the progress bar
            x, y = int(center[0]), int(center[1])
            bar_height = 10
            bar_width = 50

            # Draw the background of the bar
            cv2.rectangle(frame, (x, y - 30), (x + bar_width, y - 30 + bar_height), (0, 0, 0), -1)
            
            # Draw the progress
            progress = int((buffer_len / max_len) * bar_width)
            cv2.rectangle(frame, (x, y - 30), (x + progress, y - 30 + bar_height), (0, 255, 0), -1)

            # Draw the text indicator
            cv2.putText(frame, f"Window: {buffer_len}/{max_len}", (x, y - 35), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)

            # Display the actual window values in rows of 3
            window_values = [f"{value:.2f}" for value in object_buffers[key]]  # Display the area values
            for i in range(0, len(window_values), 3):
                row_values = " | ".join(window_values[i:i+3])
                cv2.putText(frame, row_values, (x, y - 50 - (i//3) * 15), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)

            # Perform FFT and find the dominant frequency if the buffer is full
            if buffer_len == max_len:
                func = np.array(object_buffers[key])
                fftx = np.abs(np.fft.rfft(func))
                f_axis = np.linspace(0, fps/2, int(0.5 * func.size) + 1)
                Y1 = fftx / np.max(fftx)
                x_fft = Y1[1:]
                peaks, _ = find_peaks(x_fft, height=np.max(x_fft) * 0.5)  # Finding peaks above 50% of the max

                if len(peaks) > 0:
                    dominant_frequency = f_axis[peaks[0] + 1]  # Peaks[0] is the index in the frequency domain
                    # Display the dominant frequency on the frame
                    cv2.putText(frame, f"Freq: {dominant_frequency:.2f} Hz", (x+10, y +5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

    # Display windows even for objects that have disappeared but are within the threshold
    for key, buffer in object_buffers.items():
        if len(buffer) > 0:
            # Draw window for objects not currently detected but within the threshold
            if key not in objects and frame_counter - last_seen_frame[key] <= DISAPPEARANCE_THRESHOLD:
                center = last_known_centers[key]
                x, y = int(center[0]), int(center[1])
                bar_height = 10
                bar_width = 50

                # Draw the background of the bar
                cv2.rectangle(frame, (x, y - 30), (x + bar_width, y - 30 + bar_height), (0, 0, 0), -1)
                
                # Draw the progress
                progress = int((len(buffer) / max_len) * bar_width)
                cv2.rectangle(frame, (x, y - 30), (x + progress, y - 30 + bar_height), (0, 255, 0), -1)

                # Draw the text indicator
                cv2.putText(frame, f"Window: {len(buffer)}/{max_len}", (x, y - 35), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)

                # Display the actual window values in rows of 3
                window_values = [f"{value:.2f}" for value in buffer]  # Display the area values
                for i in range(0, len(window_values), 3):
                    row_values = " | ".join(window_values[i:i+3])
                    cv2.putText(frame, row_values, (x, y - 50 - (i//3) * 15), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)

def process_frame(frame, frame_counter, object_buffers, last_seen_frame, last_known_centers):
    global next_id
    binary_frame = convert_to_binary(frame)
    contours = extract_contours(binary_frame)
    objects = frame_objects(contours, frame.shape[:2])

    features = save_object_parameters(objects)

    seen_keys = set()
    if features:  # Check if features is not empty
        for obj_features, obj in zip(features, objects):
            area = obj_features[0]
            center = obj[0][0]  # Extract the center from the object
            key = get_object_key(center, object_buffers, last_known_centers)
            if key is None:
                key = next_id
                next_id += 1
                object_buffers[key] = deque(maxlen=BUFFER_MAXLEN)
                # Create a new CSV file for the new object
                with open(f'C:\\Users\\thaim\\OneDrive\\Desktop\\Tal_Projects\\Gas_detector\\General_Codes\\Gas_Detector_5_24\\LED_FREQUENCY_TEST\\window_{key}.csv', 'w', newline='') as csvfile:
                    csvwriter = csv.writer(csvfile)
                    csvwriter.writerow([f'Frame {frame_counter}'] + [area])  # First row with the first value
            else:
                # Save the current state of the deque for each frame, even if it's not full
                with open(f'C:\\Users\\thaim\\OneDrive\\Desktop\\Tal_Projects\\Gas_detector\\General_Codes\\Gas_Detector_5_24\\LED_FREQUENCY_TEST\\window_{key}.csv', 'a', newline='') as csvfile:
                    csvwriter = csv.writer(csvfile)
                    csvwriter.writerow([f'Frame {frame_counter}'] + list(object_buffers[key]))
                
            object_buffers[key].append(area)  # Append only the area
            last_seen_frame[key] = frame_counter
            last_known_centers[key] = center  # Update the last known center
            seen_keys.add(key)

    # Add zero for missing objects up to DISAPPEARANCE_THRESHOLD
    for key in list(object_buffers.keys()):
        if key not in seen_keys:
            frames_missing = frame_counter - last_seen_frame[key]
            if frames_missing <= DISAPPEARANCE_THRESHOLD:
                object_buffers[key].append(0)  # Add zero if the object is missing
                # Update CSV file with the new zero value
                with open(f'C:\\Users\\thaim\\OneDrive\\Desktop\\Tal_Projects\\Gas_detector\\General_Codes\\Gas_Detector_5_24\\LED_FREQUENCY_TEST\\window_{key}.csv', 'a', newline='') as csvfile:
                    csvwriter = csv.writer(csvfile)
                    csvwriter.writerow([f'Frame {frame_counter}'] + list(object_buffers[key]))
            else:
                # The buffer is not deleted immediately; it's up to draw_contours to handle display logic.
                pass

    return object_buffers, objects, binary_frame

def main(video_source):
    time.sleep(0.1)
    cap = cv2.VideoCapture(video_source)
    if not cap.isOpened():
        print("Error: Could not open video source.")
        return

    frame_counter = 0
    object_buffers = {}  # Dictionary to store buffers for each object
    last_seen_frame = {}  # Dictionary to store the last frame an object was seen
    last_known_centers = {}  # Dictionary to store the last known centers of objects

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        time.sleep(0.6)
        frame_counter += 1
        object_buffers, objects, binary_frame = process_frame(frame, frame_counter, object_buffers, last_seen_frame, last_known_centers)  # Capture binary_frame
        
        # Analyze the buffers to find any frequency patterns
        # analyze_buffers(object_buffers)
        
        # Draw the contours and the window indicators
        draw_contours(frame, objects, object_buffers, frame_counter, last_seen_frame, last_known_centers)  # Pass last_known_centers here

        # Display the binary frame
        # cv2.imshow('Binary Frame', binary_frame)
        cv2.imshow('Frame', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()


In [ ]:
if __name__ == "__main__":
    video_path = r"C:\Users\thaim\Videos\IR\Recording 2024-08-18 141852 - IR_LED_1-5hz_1m.mp4"
    main(video_path)  # Use 0 for webcam, or provide a video file path

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\thaim\\OneDrive\\Desktop\\Tal_Projects\\Gas_detector\\General_Codes\\Gas_Detector_5_24\\LED_FREQUENCY_TEST\\window_0.csv'

: 

## FROM GPT

In [ ]:
import cv2
import numpy as np
from scipy.fft import fft, fftfreq
from collections import defaultdict, deque
from scipy.spatial.distance import euclidean
import time

def detect_blinking_frequency_with_fixed_window(
        video_source, expected_frequency, tolerance=2, max_distance=30, disappearance_grace=30, fft_window_size=20):
    """
    Detects the blinking frequency of individual objects in a video, adding zeros for disappeared frames.

    :param video_source: Path to the video file or integer for live camera feed.
    :param expected_frequency: Expected blinking frequency of LEDs (Hz).
    :param tolerance: Allowable deviation from expected frequency (Hz).
    :param max_distance: Maximum distance to match objects across frames.
    :param disappearance_grace: Maximum frames an object can disappear before being removed.
    :param fft_window_size: Number of frames used for FFT analysis.
    :return: Dictionary with objects' IDs and their detected frequencies.
    """
    cap = cv2.VideoCapture(video_source)
    frame_rate = int(cap.get(cv2.CAP_PROP_FPS))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    active_objects = {}  # {object_id: (x, y, brightness_history)}
    pending_objects = {}  # {object_id: (x, y, brightness_history, missing_frames)}
    object_frequencies = {}  # {object_id: detected_frequency}
    next_object_id = 0

    print(f"Frame Rate: {frame_rate} FPS")
    print(f"Total Frames: {total_frames}")

    while cap.isOpened():
        time.sleep(0.01)
        for frame_idx in range(total_frames):
            ret, frame = cap.read()
            if not ret:
                break

            # Preprocessing
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            _, thresh = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY)
            contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            # Detect objects in the current frame
            current_frame_objects = []
            for contour in contours:
                (x, y), radius = cv2.minEnclosingCircle(contour)
                if radius > 2:  # Filter out noise by size
                    brightness = np.sum(thresh[int(y - radius):int(y + radius), int(x - radius):int(x + radius)])
                    current_frame_objects.append((int(x), int(y), brightness))

            # Update active objects
            updated_objects = {}
            for x, y, brightness in current_frame_objects:
                matched_id = None
                best_match_distance = float('inf')

                # Match with active objects
                for obj_id, (prev_x, prev_y, brightness_history) in active_objects.items():
                    distance = euclidean((x, y), (prev_x, prev_y))
                    if distance < max_distance and distance < best_match_distance:
                        matched_id = obj_id
                        best_match_distance = distance

                # Match with pending objects if no active match
                if matched_id is None:
                    for obj_id, (prev_x, prev_y, brightness_history, missing_frames) in pending_objects.items():
                        distance = euclidean((x, y), (prev_x, prev_y))
                        if distance < max_distance and distance < best_match_distance:
                            matched_id = obj_id
                            best_match_distance = distance

                # Assign a new ID if no match found
                if matched_id is None:
                    matched_id = next_object_id
                    next_object_id += 1

                # Update object information
                if matched_id in pending_objects:
                    # Move from pending to active
                    _, _, brightness_history, _ = pending_objects.pop(matched_id)
                else:
                    brightness_history = deque([0] * fft_window_size, maxlen=fft_window_size)

                brightness_history.append(brightness)
                updated_objects[matched_id] = (x, y, brightness_history)

                # Visualize the object
                cv2.circle(frame, (x, y), 10, (0, 255, 0), 2)
                cv2.putText(frame, f"ID: {matched_id}", (x - 20, y - 20),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)

                # Display brightness history on the frame
                brightness_text = ', '.join(map(str, map(int, brightness_history)))
                cv2.putText(frame, f"Window: [{brightness_text}]", (x - 50, y + 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 255, 255), 1)

            # Handle objects that disappeared
            for obj_id in list(active_objects.keys()):
                if obj_id not in updated_objects:
                    if obj_id in pending_objects:
                        # Add zeros for each missing frame
                        _, _, brightness_history, missing_frames = pending_objects[obj_id]
                        brightness_history.append(0)
                        missing_frames += 1
                    else:
                        # Start tracking disappearance
                        _, _, brightness_history = active_objects[obj_id]
                        brightness_history.append(0)
                        missing_frames = 1
                    if missing_frames <= disappearance_grace:
                        pending_objects[obj_id] = (*active_objects[obj_id][:2], brightness_history, missing_frames)
                    else:
                        # Permanently remove the object
                        object_frequencies.pop(obj_id, None)

            # Update active objects for next frame
            active_objects = updated_objects

            # Show the frame with frame counter and detected objects
            cv2.putText(frame, f"Frame: {frame_idx + 1}/{total_frames}", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
            cv2.imshow("LED Frequency Detection", frame)

            if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to exit
                cap.release()
                cv2.destroyAllWindows()
                return object_frequencies
        
        # Frequency Analysis
        for obj_id, (_, _, brightness_history) in active_objects.items():
            if len(brightness_history) == fft_window_size:  # Ensure sufficient data
                fft_result = fft(list(brightness_history))
                freqs = fftfreq(len(fft_result), 1 / frame_rate)
                positive_freqs = freqs[:len(freqs) // 2]
                magnitude = np.abs(fft_result[:len(freqs) // 2])
                dominant_frequency = positive_freqs[np.argmax(magnitude)]
                object_frequencies[obj_id] = dominant_frequency

    cap.release()
    cv2.destroyAllWindows()
    return object_frequencies


# Example usage
video_path = r"C:\Users\thaim\Videos\IR\Recording 2024-08-18 141852 - IR_LED_1-5hz_1m.mp4"
expected_blinking_frequency = 3  # Hz

result = detect_blinking_frequency_with_fixed_window(video_path, expected_blinking_frequency)
for obj_id, freq in result.items():
    print(f"Object {obj_id}: Frequency {freq:.2f} Hz")


Frame Rate: 30 FPS
Total Frames: 949
